In [40]:
#Initilize imports
import pandas as pd
import numpy as np
import datetime as dt
import pathlib as path

%matplotlib inline

In [41]:
# Use Pandas, read CSVs:
# whale_returns.csv
whale_returns_dataframe = pd.read_csv(r'C:\Users\ppate\Module_4_files\Starter_Code\Resources\whale_returns.csv')
algo_returns_dataframe = pd.read_csv(r'C:\Users\ppate\Module_4_files\Starter_Code\Resources\algo_returns.csv')
sp500_history_dataframe = pd.read_csv(r'C:\Users\ppate\Module_4_files\Starter_Code\Resources\sp500_history.csv')

# Convert dates to DateTimeIndex 

# Cleaning nulls from DataFrame by dropping
csv_data = csv_data.dropna()
csv_data

# Check duplicates
csv_data.duplicated()